# ООП. Продолжение

**Задание**: 
+ Решаем [эту задачу](https://stepik.org/lesson/24462/step/9?unit=6768) и вот [эту задачу](https://stepik.org/lesson/372017/step/2?unit=359571) 
+ Полезно будет вспомнить материал предыдущих пар 
+ Если что-то не получается, скорее всего поможет прочтение комментариев к задаче, там объяснены основные непонятные моменты
+ Там можно отправлять на проверку код, не регистрируясь и не логинясь, по крайней мере у меня получалось 

In [60]:
# задача 1
import time

class Loggable:
    def log(self, msg):
        print(str(time.ctime()) + ": " + str(msg))
        
class LoggableList(list, Loggable):
    def append(self, element):
        self.log(element)
        super().append(element)

In [61]:
# задача 2
class NewInt(int):
    def repeat(self, n=2):
        return NewInt(str(self)*n)
    
    def to_bin(self):
        return NewInt(bin(self).replace('0b', ''))

In [66]:
a = NewInt(9)
print(a.repeat())  # печатает число 99
d = NewInt(a + 5)
print(d.repeat(3)) # печатает число 141414
b = NewInt(NewInt(7) * NewInt(5))
print(b.to_bin()) # печатает 100011 - двоичное представление числа 35

# Кстати, как вы думаете, что вернет данный вызов NewInt() ?


99
141414
100011


In [69]:
b = int()

In [70]:
b

0

In [67]:
a = NewInt()

In [68]:
a

0

## Абстрактные классы

**Абстрактный класс** - класс в котором есть хотя бы один абстрактный метод.   
**Абстрактный метод** - объявленный, но не реализованный метод. 

Абстрактный класс описывает интерфейс взаимодействия с дочерними классами. 

![](https://1.bp.blogspot.com/-HR3qCiRlA_E/XkAB97uh6pI/AAAAAAAACdc/AuxyFL-rnIE4uUCmzTA5DEb9fzCB9HIwgCLcBGAsYHQ/s1600/abstractcls.png)

In [71]:
from abc import ABC, ABCMeta, abstractmethod

In [81]:
class Animal(ABC): # наследование от ABC из модуля ABC
#  # или так 
# class Animal(metaclass=ABCMeta):
   
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
        
    # общий метод, который будут использовать все наследники этого класса
    def introduce(self): 
        print("Hello! My name is %s!" % self.name)
    
    # абстрактный метод, который будет необходимо переопределять для каждого подкласса
    @abstractmethod # чтобы объявить метод абстрактным используется декоратор @abstactmethod
    def sound(self):
        print('I am Animal!')

Абстрактный класс нельзя инстанциировать, обязательно нужно унаследовать и переопределить **все** абстрактные методы в дочернем классе. 

In [82]:
animal = Animal('Animal', 4, 10)

TypeError: Can't instantiate abstract class Animal with abstract methods sound

In [74]:
class Cat(Animal):
    pass

In [75]:
cat = Cat('Cat', 4, 2) # унаследовали, но не переопределили абстрактный метод

TypeError: Can't instantiate abstract class Cat with abstract methods sound

In [83]:
class Cat(Animal):
    def sound(self): # переопределяем абстрактный метод
        super().sound()
        print('Meow!')

In [84]:
class Dog(Animal):
    def sound(self): # переопределяем абстрактный метод
        super().sound()
        print('Woof!')

In [85]:
cat = Cat('Cat', 4, 2)
dog = Dog('Dog', 4, 6)

In [86]:
cat.sound()
dog.sound()

I am Animal!
Meow!
I am Animal!
Woof!


In [80]:
cat.introduce()
dog.introduce()

Hello! My name is Cat!
Hello! My name is Dog!


**Задание:**
+ У нас есть класс Settings для хранения набора настроек, вида ключ - значение (название параметра настройки - значение параметра настройки)
+ Ключи (названия параметра) обязательно должны быть строками
+ Мы не хотим,чтобы объект Settings можно было изменять после создания (добавлять и удалять ключи)

In [87]:
class Settings:

    def __init__(self, **kwargs):
        self._data = dict(**kwargs) # от нестроковых ключей нас спасает kwargs

    def get_property(self, key):
        assert isinstance(key, str)
        return self._data[key]

In [88]:
settings = Settings(setting1=1, setting2=2)

In [89]:
settings.get_property('setting1')

1

+ Допустим нам пришла в голову здравая мысль, что мы хотим вместо вызова ```settings.get_property('key')``` использовать квадратные скобки как для dict - ```settings['key']``` 
+ Ну и раз уж нас класс становится похож на readonly-dict хотелось бы чтобы он умел делать то же, что и неизменяемые словари и использоваться как словарь
+ Здесь нам поможет класс ```Mapping``` из ```collections.abc```, он покажет, какой должен быть интерфейс у нашего класса и что мы не забыли ни про какие нужные методы
+ Модуль ```collections.abc``` содержит множество абстрактных классов для создания объектов-контейнеров различных типов (словарь, список, множество, итератор и т.д.), [документация](https://docs.python.org/3/library/collections.abc.html)

In [93]:
from collections.abc import Mapping

In [98]:
# давайте отнаследуемся от класса Mapping и посмотрим, каких методов нам не хватает
class Settings(Mapping):
    def __init__(self, **kwargs):
        self._data = dict(**kwargs)
        
    def __getitem__(self, key):
        assert isinstance(key, str)
        return self._data[key]
        
    def __iter__(self):
        return self._data.__iter__()
    
    def __len__(self):
        return len(self._data)
    
    def items(self):
        return self._data.items()


Нужно сделать так, чтобы все, что написано ниже заработало:

In [99]:
settings = Settings(setting1=1, setting2=2)

print(settings['setting1']) # получение значения по ключу
print(len(settings)) # длина словаря
for setting in settings: # итерируемся по ключам
    print(setting)

1
2
setting1
setting2


In [100]:
print('setting1' in settings) # проверка наличия элемента 
print('setting3' in settings)

True
False


In [101]:
print(settings.items())

dict_items([('setting1', 1), ('setting2', 2)])


## Статические методы и методы классов

### @classmethod

Кроме полей класса, объект класса также может иметь методы класса, в которых вместо ссылки на объект экзепляра (self), передается ссылка на объект класса (cls).     

+ имеют доступ только к полям класса (но не к полям экземпляра)
+ не требуют создания экземпляра 
+ не зависят от состояния объекта

In [102]:
class Animal:
   
    fav_food = 'pizza' # атрибут класса 
    
    
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
    
    def introduce(self): 
        return "Hello! My name is %s!" % self.name
    
    
    def sound(self):
        return "Sound!"
    
    # нужно использовать декоратор classmethod
    @classmethod
    def favorite_food(cls): # cls вместо self
        return "My favorite food is %s!" % cls.fav_food

In [103]:
animal = Animal('Doggy', 4, 1)

In [104]:
animal.favorite_food() # из объекта экземпяра класса

'My favorite food is pizza!'

In [105]:
Animal.favorite_food() # из объекта класса

'My favorite food is pizza!'

**Задание**: 
+ Решаем [эту задачу](https://stepik.org/lesson/361905/step/2?unit=346443) 
+ С помощью нее поймем, как создавать методы класса и как динамически менять атрибуты класса, и зачем это все может быть нужно (чтобы хранить состояние класса)
+ Метод \_\_del\_\_ прописывать в данной задаче не нужно!
+ Там можно отправлять на проверку код, не регистрируясь и не логинясь, по крайней мере у меня получалось 


In [115]:
class Robot:
    population = 0
    
    def __init__(self, name):
        self.name = name
        Robot.population += 1
        print("Робот %s был создан" % self.name)
    
    def destroy(self):
        Robot.population -= 1
        print("Робот %s был уничтожен" % self.name)
    
    def say_hello(self):
        print("Робот %s приветствует тебя, особь человеческого рода" % self.name)
    
    @classmethod
    def how_many(cls):
        print("%s, вот сколько нас еще осталось" % cls.population)

In [112]:
# r2 = Robot("R2-D2") # печатает "Робот R2-D2 был создан"
# r2.say_hello() # печатает "Робот R2-D2 приветствует тебя, особь человеческого рода"
# Robot.how_many() # печатает "1, вот сколько нас еще осталось"
# r2.destroy() # печатает "Робот R2-D2 был уничтожен"
# Robot.how_many()

Робот R2-D2 был создан
Робот R2-D2 приветствует тебя, особь человеческого рода
1, вот сколько нас еще осталось
Робот R2-D2 был уничтожен
0, вот сколько нас еще осталось


In [113]:
del r2

In [114]:
# все работает несмотря на то, что экземпляров класса нет
Robot.how_many()

0, вот сколько нас еще осталось


### @staticmethod

+ не требуют создания экземпляра 
+ не зависят от состояния объекта
+ нет доступа даже к полям класаа
+ ничего не нужно знать про класс, пользуются только переданными аргументами
+ не нужно передавать ссылку ни на объект экземпляра, ни на объект класса

In [10]:
class Animal:
   
    fav_food = 'pizza' # атрибут класса 
    
    
    def __init__(self, name, legs, scariness):
        self.name = self.capitalize(name) 
        self.legs = legs
        self.scariness = scariness
    
    def introduce(self): 
        return "Hello! My name is %s!" % self.name
    
    
    def sound(self):
        return "Sound!"
    
    # нужно использовать декоратор staticmethod
    @staticmethod
    def capitalize(name): # аргумент self/cls не нужен, т.к. мы не обращаемся ни к полям экземпляра, ни к полям класса
        chars = list(name)
        chars[0] = chars[0].upper()
        return ''.join(chars)

In [11]:
animal = Animal('name', 4, 10)

In [12]:
animal.capitalize('name')

'Name'

In [13]:
print(animal)

## Классы данных - dataclasses

+ python 3.7+
+ автоматизируют генерацию кода классов, использующихся для хранения данных
+ без использования метклассов
+ лучше использовать классы данных чем кортеж, словарь и т.д, удобнее получать доступ к атрибутам, код красивее, понятнее и проще для тестирования и отладки
+ [документация](https://docs.python.org/3/library/dataclasses.html)

In [6]:
from dataclasses import dataclass

In [7]:
# создаем класс игральной карты

@dataclass
class Card:
    rank: str
    suit: str

Допишет за нас по умолчанию:
+ ***init*** с заданными нами аргументами
+ красивый ***repr***
+ ***eq*** (элементы равны, если равны значения всех атрибутов)

In [8]:
# init
queen_of_hearts = Card(rank='Q', suit='heart')

In [9]:
# красивый repr
queen_of_hearts

Card(rank='Q', suit='heart')

In [14]:
another_queen_of_hearts = Card(rank='Q', suit='heart')

In [15]:
# равенство
another_queen_of_hearts == queen_of_hearts

True

Без использования dataclass все то же самое выглядело бы вот так:

In [ ]:
class RegularCard:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return (f'{self.__class__.__name__}'
                f'(rank={self.rank!r}, suit={self.suit!r})')

    def __eq__(self, other):
        if other.__class__ is not self.__class__:
            return NotImplemented
        return (self.rank, self.suit) == (other.rank, other.suit)

Параметры с их значениями по умолчанию (подробнее можно почитать в документации):

In [16]:
@dataclass(init=True, # добавить init  
           repr=True, # добавить repr
           eq=True, # добавить eq
           order=False, # добавить методы для сравнения и сортировки (__le__, __ge__ и т.д.)
           unsafe_hash=False, # настройщик добавления метода __hash__
           frozen=False) # сделать класс неизменяемым
class Card:
    rank: str
    suit: str

Order - по умолчанию объекты сравниваются как кортежи из всех их атрибутов:

In [17]:
@dataclass(order=True, frozen=True)
class Card:
    rank: str
    suit: str

In [18]:
queen_hearts = Card('Q', 'hearts')
queen_spades = Card('Q', 'spades')
ten_hearts = Card('10', 'hearts')
ten_spades = Card('10', 'spades')
king_hearts = Card('K', 'hearts')
king_spades = Card('K', 'spades')

In [19]:
queen_hearts.new_attr = 10

FrozenInstanceError: cannot assign to field 'new_attr'

Вот так работает в варианте по умолчанию, но для полноценного представления колоды карт придется переписать:

In [20]:
queen_hearts > ten_spades

True

In [21]:
queen_spades > ten_hearts

True

In [22]:
queen_spades < king_spades # неправильно

False

In [23]:
for card in sorted([queen_hearts, queen_spades, ten_hearts, ten_spades, 
                    king_hearts, king_spades]):
    print(card)

Card(rank='10', suit='hearts')
Card(rank='10', suit='spades')
Card(rank='K', suit='hearts')
Card(rank='K', suit='spades')
Card(rank='Q', suit='hearts')
Card(rank='Q', suit='spades')


Также легко добавить 
+ значение атрибута по умолчанию
+ атрибуты класса

In [24]:
from typing import ClassVar

@dataclass
class Animal:
    fav_food: ClassVar = 'pizza' # переменная класса
        
    name: str
    # значения по умолчанию
    species: str = 'unknown'
    scariness: int = 0 
    legs: int = 4
    

In [25]:
cat = Animal('Kitty','Cat')

In [26]:
cat.legs

4

In [27]:
Animal.fav_food

'pizza'

**Задание:**
   + сделать рабочий датакласс карт, чтобы можно было играть в пьяницу например
   + нормальное сравнение карт
   + карты разной масти, но одинакового достоинства равны
   + порядок возможных номиналов по возрастанию: ['6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
   + класс должен быть неизменяемым

In [28]:
@dataclass(frozen=True, eq=False)
class Card:
    card_ranks: ClassVar = ['6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        
    rank: str
    suit: str
        
    def __eq__(self, other):
        return sefl.rank == other.rank
    
    def __gt__(self, other):
        return self.card_ranks.index(self.rank) > self.card_ranks.index(other.rank)
        

In [29]:
ranks = ['6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
suits = ['hearts', 'diamonds', 'spades', 'clubs']
deck = []

for suit in suits:
    for rank in ranks:
        deck.append(Card(rank=rank,suit=suit))

In [30]:
deck

[Card(rank='6', suit='hearts'),
 Card(rank='7', suit='hearts'),
 Card(rank='8', suit='hearts'),
 Card(rank='9', suit='hearts'),
 Card(rank='10', suit='hearts'),
 Card(rank='J', suit='hearts'),
 Card(rank='Q', suit='hearts'),
 Card(rank='K', suit='hearts'),
 Card(rank='A', suit='hearts'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='diamonds'),
 Card(rank='8', suit='diamonds'),
 Card(rank='9', suit='diamonds'),
 Card(rank='10', suit='diamonds'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='K', suit='diamonds'),
 Card(rank='A', suit='diamonds'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='spades'),
 Card(rank='Q', suit='spades'),
 Card(rank='K', suit='spades'),
 Card(rank='A', suit='spades'),
 Card(rank='6', suit='clubs'),
 Card(rank='7', suit='clubs'),
 Card(rank='8', suit='clubs'),
 Card(rank='9', suit='

In [31]:
sorted(deck)

[Card(rank='6', suit='hearts'),
 Card(rank='6', suit='diamonds'),
 Card(rank='6', suit='spades'),
 Card(rank='6', suit='clubs'),
 Card(rank='7', suit='hearts'),
 Card(rank='7', suit='diamonds'),
 Card(rank='7', suit='spades'),
 Card(rank='7', suit='clubs'),
 Card(rank='8', suit='hearts'),
 Card(rank='8', suit='diamonds'),
 Card(rank='8', suit='spades'),
 Card(rank='8', suit='clubs'),
 Card(rank='9', suit='hearts'),
 Card(rank='9', suit='diamonds'),
 Card(rank='9', suit='spades'),
 Card(rank='9', suit='clubs'),
 Card(rank='10', suit='hearts'),
 Card(rank='10', suit='diamonds'),
 Card(rank='10', suit='spades'),
 Card(rank='10', suit='clubs'),
 Card(rank='J', suit='hearts'),
 Card(rank='J', suit='diamonds'),
 Card(rank='J', suit='spades'),
 Card(rank='J', suit='clubs'),
 Card(rank='Q', suit='hearts'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='Q', suit='spades'),
 Card(rank='Q', suit='clubs'),
 Card(rank='K', suit='hearts'),
 Card(rank='K', suit='diamonds'),
 Card(rank='K', suit='spade